Because the intermediate are with different ways of preprocessing, I need to preprocess the data again to get the count matrix
[Follow their protocol](https://static-content.springer.com/esm/art%3A10.1038%2Fs41598-018-27293-5/MediaObjects/41598_2018_27293_MOESM1_ESM.pdf)

[Tutorial on RNA-seq, Star aligner](https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/03_alignment.html)

[How to get all SRA file at once](https://www.biostars.org/p/93494/)

[SRAdb tutorial](https://firas.io/post/sradb_part1/)

1. download
2. fastq-dump (convert sra to fastq)
3. Star aligner
4. FeatureCount
5. counfounder removal? (optional)

In [2]:
library(readxl)
geo = read_excel('RNA-seq.xlsx')

New names:
* `` -> ...1


In [3]:
geo

...1,GEO accession,species,SRA
Darmanis,GSE67835,human,SRP057196
Zhang,GSE73721,human,SRP064454
Zhang,GSE52564,mouse,SRP033200
Zeisel,GSE60361,mouse,SRP045452
Tasic,GSE71585,mouse,SRP061902


In [4]:
target_directory = '/cellar/users/hsher/Data/ideker/sc/'
library(SRAdb)
srafile = '/cellar/users/hsher/ontoencoder/data/SRAmetadb.sqlite'


Loading required package: RSQLite
Loading required package: graph
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: RCurl
Loading required

In [5]:
sra_con <- dbConnect(SQLite(),srafile)

In [6]:
#"ascp -T -l 300m -i /cellar/users/hsher/.aspera/connect/etc/asperaweb_id_dsa.openssh" -> ascpCMD
#getFASTQfile(geo$SRA, sra_con, destDir = target_directory, srcType = "fasp", ascpCMD = ascpCMD,  makeDirectory = TRUE)

#getSRAfile(geo$SRA, sra_con, fileType = 'fastq', destDir = target_directory, srcType = "fasp", ascpCMD = ascpCMD)

In [ ]:
for (srp in as.matrix(geo$SRA)){
    # retrieve srr
    print(srp)
    sra_info = getFASTQinfo(srp,sra_con)
    
    # retrieve fastq
    new_dir = paste(target_directory, srp, sep = '')
    print(new_dir)
    system(paste("mkdir ", new_dir))
    
    # getSRAfile( c(sra_info$run), sra_con, fileType ='fastq', destDir = new_dir)
    # getSRAfile is not robust, it works sometimes and don't work the other time
    # use bash
    # system(paste("cd", new_dir))
    
    for (url in sra_info$ftp){
        url_split = as.matrix(unlist(strsplit(url, '/')))
        filename = url_split[length(url_split)]
        #print(filename)
        destfile = paste(paste(new_dir, '/', sep = ''), filename, sep='')
        if(!file.exists(destfile)){
            system(paste(paste("wget", url), paste('-P ', new_dir)))
            print(paste(destfile, 'dont exist'))
        }
        else{
            #print(paste(destfile, 'Exist'))
        }
        
        
    }
    
}